In [13]:
import pandas as pd
import os
import json
import numpy as np
import re

In [14]:
## reading the Survey File
#surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\Test.csv",sep="," , skiprows=1)
surveyData = pd.read_csv("C:\Projects\MSS\Loading _AmenityData_from_Survey\AmenityClubSurvey FINAL v2 Season Cleaned_deletedFirstTwoColumn.csv",sep="," , skiprows=1, keep_default_na=False)
surveyData.head()

## Reading the Config File and Extracting the Information from there
config_file = open("C:\Projects\MSS\Loading _AmenityData_from_Survey\Python_Codes\ClubCorp\SteamRoom\SteamRoomColMapping.json")
FitnessCenter_config = json.load(config_file)

columnName = ""

In [15]:
def addToDayTimeMap (phonebook, key, value):
    if key in phonebook:
        x =  phonebook[key]
        phonebook[key] = x + "," + value
        
    else :
        phonebook[key] = value

In [16]:
def prepareHoursOfOperationJson(hoursOfOperationInfo, surveyData):
    
    allHoursOfOperation = []
    
    for i in range(0,len(surveyData)) : 
        hoursOfOperationForEachRow = []
        hoursOfOperation = {}
        hoursOfOperationValue = {}
        
        
        ## creating day time map so that days with same open and closed time can handled properly
        dayTimeMap = {}    
        for j in range(0, len(hoursOfOperationInfo)):
            
            dayInfo = hoursOfOperationInfo[j]
            day = dayInfo["Day"]
            openTimeIndex = dayInfo["OpenTime"]
            closeTimeIndex = dayInfo["CloseTime"]
                
            startTimeRaw = surveyData.iat[i, openTimeIndex]
            endTimeRaw = surveyData.iat[i, closeTimeIndex]
            
            
            startTime = ""
            endTime = ""
            if pd.notnull(startTimeRaw) and startTimeRaw != "":
                startTime = str(startTimeRaw).strip()
                
            
            if pd.notnull(endTimeRaw) and endTimeRaw!= "":
                endTime = str(endTimeRaw).strip()
                
            
            
            if startTime == "Closed" or endTime == "Closed" :
                continue
            else : 
                if startTime != "" or endTime != "":
                    addToDayTimeMap(dayTimeMap, startTime + "#$" + endTime, day)
            
        
        dayAndTimingArray = []
        
        for startAndEndTime, days in dayTimeMap.items():
            
            startTime,endTime = startAndEndTime.split("#$")
            openDays = days.split(",")
            ## Adding Day and Timing
            dayAndTiming = {}
            dayAndTimingValue = {}
            for day in openDays :
                dayValue = {}
                dayValue["value"] = True
                dayAndTimingValue[day] = [dayValue]

            timing = {}
            timingValue = {}

            if startTime and not pd.isnull(startTime): 
                startTimeDict ={}
                startTimeDict["value"] = startTime
                timingValue["StartTime"] =  [startTimeDict]

            if endTime and not pd.isnull(endTime):
                endTimeDict ={}
                endTimeDict["value"] = endTime
                timingValue["EndTime"] =  [endTimeDict]
            
            ## Adding day part as "All Day"
            dayPart ={}
            dayPart["value"] = "All Day"
            timingValue["DayPart"] =  [dayPart]


            if timingValue :
                timing["value"] = timingValue
                dayAndTimingValue["Timing"] = [timing]

            if dayAndTimingValue : 
                dayAndTiming["value"] = dayAndTimingValue    
                dayAndTimingArray.append(dayAndTiming)
        
        if dayAndTimingArray :
            hoursOfOperationValue["DayAndTiming"] = dayAndTimingArray
            season = "All Season"
            ## adding season value
            if season and not pd.isnull(season):
                seasonValue = {}
                seasonValue["value"] = season
                hoursOfOperationValue["Season"] = [seasonValue]
        
        if hoursOfOperationValue :
            hoursOfOperation["value"] = hoursOfOperationValue        
            hoursOfOperationForEachRow.append(hoursOfOperation)
            
            
        allHoursOfOperation.append(json.dumps(hoursOfOperationForEachRow))
    
    return allHoursOfOperation

In [17]:
def determineBooleanValues (dataFrame):
    val = False
    if pd.notnull(dataFrame[columnName]) :
        if dataFrame[columnName] == columnName:
            val = True
    return val

In [18]:
## Getting the output File name
output_fileName = FitnessCenter_config['outputFileName']


all_column_names = []
FitnessCenter_related_column_names = []

columnIndexForFitnessCenter = []

common_column_count = len(FitnessCenter_config['CommonColumn'])

for i in range(0, common_column_count):
    columnIndexForFitnessCenter.append(FitnessCenter_config['CommonColumn'][i]['columnIndex'])
    all_column_names.append(FitnessCenter_config['CommonColumn'][i]['outputColumnName'])

FitnessCenter_column_count = len(FitnessCenter_config['SteamRoomRelatedColumn'])

for j in range(0, FitnessCenter_column_count):
    columnIndexForFitnessCenter.append(FitnessCenter_config['SteamRoomRelatedColumn'][j]['columnIndex'])
    all_column_names.append(FitnessCenter_config['SteamRoomRelatedColumn'][j]['outputColumnName'])
    FitnessCenter_related_column_names.append(FitnessCenter_config['SteamRoomRelatedColumn'][j]['outputColumnName'])

## Data for Fitness Center
FitnessCenter = surveyData.iloc[:, columnIndexForFitnessCenter]
FitnessCenter.columns = all_column_names



filterCondition  = FitnessCenter['SteamRoomConditionColumn'] != 'N/A'

clubsNotHavingFitnessCenter = FitnessCenter[~filterCondition]

clubsNotHavingFitnessCenter.to_csv("C:/Projects/MSS/Loading _AmenityData_from_Survey/Python_Codes/ClubCorp/SteamRoom/SteamRoom_ClubsNotHaving.csv")



FitnessCenter = FitnessCenter[filterCondition]
FitnessCenter['Index']=1
FitnessCenter['Status'] = 'Active'
FitnessCenter['Type'] = 'Health & Wellness'
FitnessCenter['Name'] = 'Steam Room'

for x  in FitnessCenter_related_column_names:
    columnName = x
    FitnessCenter[columnName + '_Boolean'] = FitnessCenter.apply(determineBooleanValues, axis=1)


FitnessCenter.to_csv("C:\\Projects\\MSS\\Loading _AmenityData_from_Survey\\Python_Codes\\ClubCorp\\SteamRoom\\" + output_fileName)